# CNN Model F (Big Data)

- Change last Conv2D from AvgPooling2D to Conv2D, Reduce Learning-Rate

In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
# from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))
ds_xc = ds_x.map(lambda x: tf.add(x, tf.random.normal(tf.shape(x), 0, 0.1)))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_xcy = tf.data.Dataset.zip((ds_xc, ds_y))
ds_xcy = ds_xcy.shuffle(BATCH_SIZE*2)

In [8]:
ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_trainc = ds_xcy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [9]:
data_aug = Sequential([
    RandomFlip("horizontal_and_vertical"),
    # RandomZoom(0.2, 0.2)
])

In [10]:
model = Sequential()

model.add(Input(shape=(IMG_SHAPE)))
model.add(data_aug)

model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(AveragePooling2D (pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(16, (1, 1), padding='same', activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.35))

model.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(1e-4)))

# model.summary()

In [11]:
ds_trainc = ds_trainc.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=60,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/60
3095/3095 - 158s - loss: 0.4663 - accuracy: 0.8195 - val_loss: 1.3248 - val_accuracy: 0.7044
Epoch 2/60
3095/3095 - 156s - loss: 0.3661 - accuracy: 0.8633 - val_loss: 1.1251 - val_accuracy: 0.7354
Epoch 3/60
3095/3095 - 156s - loss: 0.3298 - accuracy: 0.8802 - val_loss: 1.1260 - val_accuracy: 0.7540
Epoch 4/60
3095/3095 - 156s - loss: 0.3041 - accuracy: 0.8912 - val_loss: 0.7297 - val_accuracy: 0.7901
Epoch 5/60
3095/3095 - 156s - loss: 0.2876 - accuracy: 0.8979 - val_loss: 0.6760 - val_accuracy: 0.8102
Epoch 6/60
3095/3095 - 156s - loss: 0.2730 - accuracy: 0.9051 - val_loss: 0.9363 - val_accuracy: 0.7811
Epoch 7/60
3095/3095 - 156s - loss: 0.2603 - accuracy: 0.9096 - val_loss: 0.9961 - val_accuracy: 0.7710
Epoch 8/60
3095/3095 - 156s - loss: 0.2476 - accuracy: 0.9158 - val_loss: 0.6434 - val_accuracy: 0.8135
Epoch 9/60
3095/3095 - 156s - loss: 0.2374 - accuracy: 0.9205 - val_loss: 0.7640 - val_accuracy: 0.7927
Epoch 10/60
3095/3095 - 156s - loss: 0.2298 - accuracy: 0.9237 -

In [12]:
model.save('CNN_ModelF')

INFO:tensorflow:Assets written to: CNN_ModelF/assets


In [ ]:
ds_trainc = ds_trainc.unbatch()
ds_test = ds_test.unbatch()
ds_trainc = ds_trainc.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_trainc,
                    epochs=25,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/25
1548/1548 - 155s - loss: 0.1272 - accuracy: 0.9640 - val_loss: 0.3418 - val_accuracy: 0.9081
Epoch 2/25
1548/1548 - 154s - loss: 0.1282 - accuracy: 0.9633 - val_loss: 0.5060 - val_accuracy: 0.8866
Epoch 3/25
1548/1548 - 154s - loss: 0.1269 - accuracy: 0.9638 - val_loss: 0.4448 - val_accuracy: 0.9018
Epoch 4/25
1548/1548 - 154s - loss: 0.1265 - accuracy: 0.9643 - val_loss: 0.3176 - val_accuracy: 0.9154
Epoch 5/25
1548/1548 - 154s - loss: 0.1257 - accuracy: 0.9642 - val_loss: 0.2453 - val_accuracy: 0.9309
Epoch 6/25
1548/1548 - 154s - loss: 0.1256 - accuracy: 0.9644 - val_loss: 0.2132 - val_accuracy: 0.9430
Epoch 7/25
1548/1548 - 154s - loss: 0.1240 - accuracy: 0.9652 - val_loss: 0.4741 - val_accuracy: 0.8846
Epoch 8/25
1548/1548 - 154s - loss: 0.1241 - accuracy: 0.9646 - val_loss: 0.2699 - val_accuracy: 0.9309
Epoch 9/25
1548/1548 - 154s - loss: 0.1236 - accuracy: 0.9649 - val_loss: 0.3543 - val_accuracy: 0.9109
Epoch 10/25
1548/1548 - 154s - loss: 0.1236 - accuracy: 0.9650 -

In [12]:
# ds_trainc = ds_trainc.unbatch()
# ds_test = ds_test.unbatch()
ds_trainc = ds_trainc.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_trainc,
                    epochs=25,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/25
774/774 - 162s - loss: 0.1235 - accuracy: 0.9656 - val_loss: 0.3469 - val_accuracy: 0.9165
Epoch 2/25
774/774 - 154s - loss: 0.1214 - accuracy: 0.9664 - val_loss: 0.5145 - val_accuracy: 0.8838
Epoch 3/25
774/774 - 154s - loss: 0.1208 - accuracy: 0.9665 - val_loss: 0.5481 - val_accuracy: 0.8743
Epoch 4/25
774/774 - 154s - loss: 0.1187 - accuracy: 0.9670 - val_loss: 0.3956 - val_accuracy: 0.9106
Epoch 5/25
774/774 - 154s - loss: 0.1179 - accuracy: 0.9671 - val_loss: 0.4080 - val_accuracy: 0.9046
Epoch 6/25
774/774 - 154s - loss: 0.1173 - accuracy: 0.9679 - val_loss: 0.3767 - val_accuracy: 0.9139
Epoch 7/25
774/774 - 154s - loss: 0.1167 - accuracy: 0.9675 - val_loss: 0.4505 - val_accuracy: 0.8954
Epoch 8/25
774/774 - 154s - loss: 0.1160 - accuracy: 0.9680 - val_loss: 0.3730 - val_accuracy: 0.9087
Epoch 9/25
774/774 - 154s - loss: 0.1145 - accuracy: 0.9686 - val_loss: 0.3508 - val_accuracy: 0.9134
Epoch 10/25
774/774 - 154s - loss: 0.1151 - accuracy: 0.9675 - val_loss: 0.4198 - 

In [13]:
model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_trainc,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 155s - loss: 0.1009 - accuracy: 0.9727 - val_loss: 0.2747 - val_accuracy: 0.9291
Epoch 2/10
774/774 - 154s - loss: 0.0986 - accuracy: 0.9735 - val_loss: 0.2717 - val_accuracy: 0.9299
Epoch 3/10
774/774 - 154s - loss: 0.0978 - accuracy: 0.9737 - val_loss: 0.2686 - val_accuracy: 0.9306
Epoch 4/10
774/774 - 154s - loss: 0.0979 - accuracy: 0.9736 - val_loss: 0.2681 - val_accuracy: 0.9300
Epoch 5/10
774/774 - 154s - loss: 0.0975 - accuracy: 0.9738 - val_loss: 0.2668 - val_accuracy: 0.9313
Epoch 6/10
774/774 - 154s - loss: 0.0971 - accuracy: 0.9737 - val_loss: 0.2909 - val_accuracy: 0.9254
Epoch 7/10
774/774 - 154s - loss: 0.0967 - accuracy: 0.9740 - val_loss: 0.2819 - val_accuracy: 0.9271
Epoch 8/10
774/774 - 154s - loss: 0.0960 - accuracy: 0.9743 - val_loss: 0.2781 - val_accuracy: 0.9285
Epoch 9/10
774/774 - 154s - loss: 0.0961 - accuracy: 0.9743 - val_loss: 0.2820 - val_accuracy: 0.9288
Epoch 10/10
774/774 - 154s - loss: 0.0965 - accuracy: 0.9742 - val_loss: 0.2884 - 

In [14]:
model.save('CNN_ModelF')

INFO:tensorflow:Assets written to: CNN_ModelF/assets


In [15]:
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)

history = model.fit(ds_train,
                    epochs=25,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/25
774/774 - 154s - loss: 0.0842 - accuracy: 0.9792 - val_loss: 0.0952 - val_accuracy: 0.9761
Epoch 2/25
774/774 - 154s - loss: 0.0823 - accuracy: 0.9796 - val_loss: 0.0934 - val_accuracy: 0.9763
Epoch 3/25
774/774 - 154s - loss: 0.0818 - accuracy: 0.9800 - val_loss: 0.0960 - val_accuracy: 0.9756
Epoch 4/25
774/774 - 154s - loss: 0.0808 - accuracy: 0.9803 - val_loss: 0.0961 - val_accuracy: 0.9757
Epoch 5/25
774/774 - 154s - loss: 0.0816 - accuracy: 0.9801 - val_loss: 0.0944 - val_accuracy: 0.9768
Epoch 6/25
774/774 - 154s - loss: 0.0804 - accuracy: 0.9802 - val_loss: 0.0939 - val_accuracy: 0.9767
Epoch 7/25
774/774 - 154s - loss: 0.0799 - accuracy: 0.9804 - val_loss: 0.0929 - val_accuracy: 0.9769
Epoch 8/25
774/774 - 154s - loss: 0.0795 - accuracy: 0.9805 - val_loss: 0.0927 - val_accuracy: 0.9768
Epoch 9/25
774/774 - 154s - loss: 0.0800 - accuracy: 0.9803 - val_loss: 0.0933 - val_accuracy: 0.9770
Epoch 10/25
774/774 - 154s - loss: 0.0789 - accuracy: 0.9809 - val_loss: 0.0958 - 

In [16]:
model.save('CNN_ModelF')

INFO:tensorflow:Assets written to: CNN_ModelF/assets
